In [1]:
import numpy as np
import torch.nn as nn
import torch
import numpy as np
from transformers import AutoTokenizer
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [2]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('sentence-transformers/paraphrase-TinyBERT-L6-v2')

In [13]:
model = BertModel.from_pretrained('sentence-transformers/paraphrase-TinyBERT-L6-v2')

In [16]:
#load the model weights from the file via loading the state_dict
model.load_state_dict(torch.load('model_weights.pth'))

<All keys matched successfully>

In [3]:
text = "Here is the sentence I want embeddings for."
tokenized_text = tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt')

In [7]:
embedded_text = model(**tokenized_text)

In [9]:
embedded_text.last_hidden_state

tensor([[[-0.0936,  0.0594, -0.2101,  ..., -0.2028, -0.3142,  0.1470],
         [ 0.0616,  0.1545, -0.0600,  ..., -0.0056, -0.4102,  0.0247],
         [-0.2283,  0.1356, -0.0914,  ..., -0.0470, -0.1464, -0.1197],
         ...,
         [ 0.2567, -0.0418, -0.0823,  ..., -0.0983, -0.3467,  0.0449],
         [-0.1123, -0.1109, -0.2205,  ..., -0.2437, -0.1630, -0.0140],
         [-0.0598,  0.0138,  0.2487,  ..., -0.0523, -0.1540,  0.2993]]],
       grad_fn=<NativeLayerNormBackward0>)

In [18]:
tokenized_text

{'input_ids': tensor([[ 101, 2182, 2003, 1996, 6251, 1045, 2215, 7861, 8270, 4667, 2015, 2005,
         1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [17]:
embedded_text = model(**tokenized_text)

In [19]:
embedded_text.last_hidden_state

tensor([[[-0.0936,  0.0594, -0.2101,  ..., -0.2028, -0.3142,  0.1470],
         [ 0.0616,  0.1545, -0.0600,  ..., -0.0056, -0.4102,  0.0247],
         [-0.2283,  0.1356, -0.0914,  ..., -0.0470, -0.1464, -0.1197],
         ...,
         [ 0.2567, -0.0418, -0.0823,  ..., -0.0983, -0.3467,  0.0449],
         [-0.1123, -0.1109, -0.2205,  ..., -0.2437, -0.1630, -0.0140],
         [-0.0598,  0.0138,  0.2487,  ..., -0.0523, -0.1540,  0.2993]]],
       grad_fn=<NativeLayerNormBackward0>)

Masz tutaj script o oktorym w komicie wspominalem

In [20]:
import translators as ts
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('sentence-transformers/paraphrase-TinyBERT-L6-v2')
model = BertModel.from_pretrained('sentence-transformers/paraphrase-TinyBERT-L6-v2')
#load the model weights from the file via loading the state_dict
model.load_state_dict(torch.load('model_weights.pth'))

<All keys matched successfully>

In [26]:
#script that translates the text from polish to english and then tokenizes and embeds it
def translate_token_embed(text):
    translated_text = ts.translate_text(query_text=text, translator='google', from_language='pl', to_language='en')
    tokenized_text = tokenizer.encode_plus(translated_text, add_special_tokens=True, return_tensors='pt')
    embedded_text = model(**tokenized_text)
    return embedded_text.last_hidden_state
